# Extract Faces

## Set-up

In [0]:
#@title Run in google colab?
# this is to make use of gpu on colab.
# set to false if running locally on jupyter notebook
google_colab = True #@param {type:"boolean"}

In [0]:
# mount google drive if running from colab
if google_colab:
    from google.colab import drive
    drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# install facenet_pytorch if working in colab
if google_colab:
    !pip install facenet-pytorch

     |████████████████████████████████| 1.9MB 12.2MB/s 


In [0]:
#import relevant libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import datasets
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
from PIL import Image
import time
import os
import glob
import shutil
import imageio
from sklearn.cluster import KMeans
from scipy import stats

In [0]:
# move to same directory where this notebook is (if using colab)
if google_colab:
    os.chdir('/content/drive/My Drive/MSc Data Science/Computer Vision/Coursework/code')

In [0]:
# import utils package with custom methods for this coursework
import utils

In [0]:
# Set folder paths to fetch images and save extracted faces 
# (use relative paths from where this notebook is saved)
images_i_path = '../data/individual images' 
images_g_path = '../data/group images/jpeg' 
train_path = '../data/train' 

In [0]:
#check gpu is enabled
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [0]:
# delete previously extracted faces if required
if os.path.exists(train_path):
        shutil.rmtree(train_path)
else:
    os.makedirs(train_path)

# create faces directory (if none exists)
if not os.path.exists(train_path):
    os.makedirs(train_path)

In [0]:
# create mtcnn face detection model
# sourced from facenet-pytorch library,
# more info here: https://github.com/timesler/facenet-pytorch
mtcnn = MTCNN(select_largest=False, post_process=False, device=device) 

## Extract faces from images

### Extract faces

In [0]:
start_time = time.time()
labels = [file.name for file in os.scandir(images_i_path) if file.is_dir()]
faces_dict = {label: [] for label in labels}
# loop through label directories
for label in labels:
    print('\n\n',f'label {label}:','\n')
    # pick up jpg file names in directory
    image_names = [image_name.split('/')[-1] for image_name in glob.glob(f'{images_i_path}/{label}/*.JPG')]
    print(f'images found: {len(image_names)}')
    # create empty list to hold loaded images
    images = []
    # loop through each image
    for i, image_name in enumerate(image_names):
        i += 1
        print(f'({i}) {image_name}')
        # read in image
        image = cv2.imread(f'{images_i_path}/{label}/{image_name}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #append to images list
        images.append(Image.fromarray(image))
    # apply face extraction on images
    faces = mtcnn(images)
    # filter out instances where no face extracted
    faces = [face for face in faces if face is not None]
    print(f'faces extracted: {len(faces)}')
    # convert faces to [160, 160, 3] numpy arrays (standard format for colour photos)
    faces = [face.permute(1,2,0).int().numpy().astype(np.uint8) for face in faces]
    # save faces in faces_dict under relevant label
    faces_dict[label].extend(faces)
    # show time elapsed
    print('\n',f'time elapsed: {round((time.time() - start_time)/60,2)} mins')

# Remove mtcnn to reduce GPU memory usage
del mtcnn
if device == torch.device('cuda:0'):
    torch.cuda.empty_cache()



 label 01: 

images found: 7
(1) IMG_6862.JPG
(2) IMG_6861.JPG
(3) IMG_6860.JPG
(4) IMG_6858.JPG
(5) IMG_6857.JPG
(6) IMG_6859.JPG
(7) IMG_6856.JPG
faces extracted: 7

 time elapsed: 0.16 mins


 label 02: 

images found: 8
(1) IMG_2598.JPG
(2) IMG_2596.JPG
(3) IMG_2597.JPG
(4) IMG_2600.JPG
(5) IMG_2595.JPG
(6) IMG_2594.JPG
(7) IMG_2599.JPG
(8) IMG_2630.JPG
faces extracted: 8

 time elapsed: 0.33 mins


 label 03: 

images found: 7
(1) IMG_6866.JPG
(2) IMG_6863.JPG
(3) IMG_6867.JPG
(4) IMG_6868.JPG
(5) IMG_6869.JPG
(6) IMG_6865.JPG
(7) IMG_6864.JPG
faces extracted: 7

 time elapsed: 0.47 mins


 label 04: 

images found: 6
(1) IMG_2623.JPG
(2) IMG_2624.JPG
(3) IMG_2629.JPG
(4) IMG_2628.JPG
(5) IMG_2626.JPG
(6) IMG_2627.JPG
faces extracted: 6

 time elapsed: 0.58 mins


 label 05: 

images found: 7
(1) IMG_6874.JPG
(2) IMG_6870.JPG
(3) IMG_6871.JPG
(4) IMG_6876.JPG
(5) IMG_6872.JPG
(6) IMG_6875.JPG
(7) IMG_6873.JPG
faces extracted: 7

 time elapsed: 0.72 mins


 label 06: 

images fou

### Save faces

In [0]:
# loop over labels in faces dict
for label in labels:
    print('\n\n',f'label {label}:','\n')
    # create label directory to save faces (if it doesn't already exist)
    label_path = train_path + f'/{label}'
    if not os.path.exists(label_path):
        os.makedirs(label_path)
    # loop over faces and save each one as jpg, with name reflecting label
    for i, face in enumerate(faces_dict[label]):
        # save face as jpg
        imageio.imwrite(f'{label_path}/{label}_i_{i}.JPG', face)
        print(f'{label}_i_{i}.JPG saved')



 label 01: 

01_i_0.JPG saved
01_i_1.JPG saved
01_i_2.JPG saved
01_i_3.JPG saved
01_i_4.JPG saved
01_i_5.JPG saved
01_i_6.JPG saved


 label 02: 

02_i_0.JPG saved
02_i_1.JPG saved
02_i_2.JPG saved
02_i_3.JPG saved
02_i_4.JPG saved
02_i_5.JPG saved
02_i_6.JPG saved
02_i_7.JPG saved


 label 03: 

03_i_0.JPG saved
03_i_1.JPG saved
03_i_2.JPG saved
03_i_3.JPG saved
03_i_4.JPG saved
03_i_5.JPG saved
03_i_6.JPG saved


 label 04: 

04_i_0.JPG saved
04_i_1.JPG saved
04_i_2.JPG saved
04_i_3.JPG saved
04_i_4.JPG saved
04_i_5.JPG saved


 label 05: 

05_i_0.JPG saved
05_i_1.JPG saved
05_i_2.JPG saved
05_i_3.JPG saved
05_i_4.JPG saved
05_i_5.JPG saved
05_i_6.JPG saved


 label 06: 

06_i_0.JPG saved
06_i_1.JPG saved
06_i_2.JPG saved
06_i_3.JPG saved
06_i_4.JPG saved
06_i_5.JPG saved
06_i_6.JPG saved
06_i_7.JPG saved


 label 07: 

07_i_0.JPG saved
07_i_1.JPG saved
07_i_2.JPG saved
07_i_3.JPG saved
07_i_4.JPG saved
07_i_5.JPG saved
07_i_6.JPG saved


 label 08: 

08_i_0.JPG saved
08_i_1.JPG sa

## Extract faces from group photos

### Extract faces

In [0]:
# create mtcnn face detection model
mtcnn = MTCNN(select_largest=False, post_process=False, device=device, keep_all=True) 

In [0]:
# pick up jpg file names in directory
image_names = [image_name.split('/')[-1] for image_name in glob.glob(f'{images_g_path}/*.JPG')]
# create empty list to hold loaded images
images = []
# loop through each image
for image_name in image_names:
    # read in image
    image = cv2.imread(f'{images_g_path}/{image_name}', cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #append to images list
    images.append(Image.fromarray(image))
# apply face extraction on images
faces_l = []
for i, image in enumerate(images):
    faces = [face.permute(1,2,0).int().numpy().astype('uint8') for face in mtcnn(image)]
    faces_l.extend(faces)
# delete mtcnn to free up gpu memory (if colab being used)
del mtcnn
if device == torch.device('cuda:0'):
    torch.cuda.empty_cache()

### Label faces

In [0]:
# instantiate facenet pre-trained model to convert faces to 512-dim image embeddings
# pre-trained model sourced from facenet-pytorch library,
# more info here: https://github.com/timesler/facenet-pytorch
facenet = InceptionResnetV1(pretrained='vggface2').to(device).eval()

In [0]:
# create empty list to hold embeddings
embeddings = []
# loop over faces, convert to embeddings with facenet
transform = transforms.Compose([transforms.ToTensor()])
for face in faces_l:
    input_ = transform(Image.fromarray(face)).unsqueeze(0).to(device)
    embedding = facenet(input_).squeeze().cpu().detach().numpy()
    embeddings.append(embedding)
    
embeddings = np.stack(embeddings)

In [0]:
# use kmeans to cluster embeddings
n_clusters = 48
kmeans = KMeans(n_clusters=n_clusters)
clusters = kmeans.fit_predict(embeddings)

In [0]:
# group together faces based on embedding clusters
faces_dict = {}
for cluster in np.arange(n_clusters):
    faces_in_cluster = list(np.array(faces_l)[clusters == cluster])
    faces_dict[cluster] = faces_in_cluster

In [0]:
# load labelled faces (from individual images)
faces = datasets.ImageFolder(train_path)
faces.idx_to_class = {i:c for c, i in faces.class_to_idx.items()}

In [0]:
labels_to_clusters = {}
for label in range(len(faces.classes)):
    faces_with_label = [face for (face, face_label) in faces if face_label==label]
    # convert faces to embeddings
    embeddings = []
    for face in faces_with_label:
        input_ = transform(face).unsqueeze(0).to(device)
        embedding = facenet(input_).squeeze().cpu().detach().numpy()
        embeddings.append(embedding)
    # use kmeans to map embeddings to clusters
    clusters = kmeans.predict(embeddings)
    # get most frequently occurring cluster (mode)
    cluster = stats.mode(clusters)[0].item()
    # map label to cluster
    labels_to_clusters[label] = cluster 

### Save faces

In [0]:
# loop over labels in faces dict
for label in range(len(faces.classes)):
    label_class = faces.idx_to_class[label]
    print('\n\n',f'label {label_class}:','\n')
    # create label directory to save faces (if it doesn't already exist)
    label_path = train_path + f'/{label_class}'
    if not os.path.exists(label_path):
        os.makedirs(label_path)
    # loop over faces and save each one as jpg, with name reflecting label
    for i, face in enumerate(faces_dict[labels_to_clusters[label]]):
        # save face as jpg
        imageio.imwrite(f'{label_path}/{label_class}_g_{i}.JPG', face)
        print(f'{label_class}_g_{i}.JPG saved')



 label 01: 

01_g_0.JPG saved
01_g_1.JPG saved
01_g_2.JPG saved
01_g_3.JPG saved
01_g_4.JPG saved
01_g_5.JPG saved
01_g_6.JPG saved
01_g_7.JPG saved
01_g_8.JPG saved
01_g_9.JPG saved
01_g_10.JPG saved
01_g_11.JPG saved
01_g_12.JPG saved
01_g_13.JPG saved
01_g_14.JPG saved


 label 02: 

02_g_0.JPG saved
02_g_1.JPG saved
02_g_2.JPG saved
02_g_3.JPG saved
02_g_4.JPG saved
02_g_5.JPG saved
02_g_6.JPG saved
02_g_7.JPG saved
02_g_8.JPG saved
02_g_9.JPG saved
02_g_10.JPG saved
02_g_11.JPG saved
02_g_12.JPG saved
02_g_13.JPG saved


 label 03: 

03_g_0.JPG saved
03_g_1.JPG saved
03_g_2.JPG saved
03_g_3.JPG saved
03_g_4.JPG saved
03_g_5.JPG saved
03_g_6.JPG saved
03_g_7.JPG saved
03_g_8.JPG saved
03_g_9.JPG saved
03_g_10.JPG saved
03_g_11.JPG saved
03_g_12.JPG saved
03_g_13.JPG saved
03_g_14.JPG saved
03_g_15.JPG saved


 label 04: 

04_g_0.JPG saved
04_g_1.JPG saved
04_g_2.JPG saved
04_g_3.JPG saved
04_g_4.JPG saved
04_g_5.JPG saved
04_g_6.JPG saved
04_g_7.JPG saved
04_g_8.JPG saved
04_g_9.

**Note:** Some faces may have been incorrectly labelled, so a manual check is required.